In [1]:
import matplotlib.pyplot as plt
%matplotlib notebook
import matplotlib.patches as mpatches
import numpy as np
import math

import mahotas as mh
import cv2
from PIL import Image

import copy, os

from skimage import io, exposure
from skimage.segmentation import clear_border
from skimage.measure import label, regionprops
from skimage.morphology import closing, opening, square, disk
from skimage.draw import disk as pdisk
from skimage.color import label2rgb, rgb2gray

from scipy import ndimage as ndi

from skimage.morphology import watershed
from skimage.feature import peak_local_max

from skimage.filters import threshold_local, threshold_otsu, threshold_multiotsu, rank
from skimage.filters import sobel

from scipy.stats import skew
from scipy.signal import argrelextrema


/home/user/.local/lib/python3.8/site-packages/skimage/io/manage_plugins.py:23: UserWarning: Your installed pillow version is < 7.1.0. Several security issues (CVE-2020-11538, CVE-2020-10379, CVE-2020-10994, CVE-2020-10177) have been fixed in pillow 7.1.0 or higher. We recommend to upgrade this library.
  from .collection import imread_collection_wrapper


In [2]:
PRECISION = 50

In [3]:
class IegImgParams(): 
    def __init__(self, path, thre, skewness, bgr):
        self.path = path
        self.thre = thre
        self.skewness = skewness
        self.bgr = bgr

In [4]:
class Approximation(): 
    def __init__(self, reg_props, approx_patch, intensity,  idno):
        self.reg_props = reg_props
        self.approx_patch = approx_patch
        self.intensity = intensity
        self.idno = idno

In [5]:
def background_filter(flattened_img):
    hist, bins = np.histogram(flattened_img, int(pow(PRECISION, 2)/20))
    mode_idx = np.argmax(hist)
    mode = bins[mode_idx]
    return mode

In [6]:
def reflect(image_ts):
    height = image_ts.shape[0]
    image_ts = np.transpose(image_ts)
    to_ret = np.copy(image_ts)
    for col_no, column in enumerate(image_ts):
        nonzero = False
        idx = 0
        while not nonzero and idx < height:
            nonzero = column[idx] > 0
            idx += 1
        start_idx = idx - 1
        while nonzero and idx < height:
            nonzero = column[idx] > 0
            idx += 1
        stop_idx = idx - 1
        reflection = np.flip(column[start_idx:stop_idx])
        reflect_offset_up = start_idx - len(reflection)
        reflect_offset_down = stop_idx + len(reflection)
        to_ret[col_no][max(0, reflect_offset_up):start_idx] = reflection[max(0, -reflect_offset_up):]
        to_ret[col_no][stop_idx:min(reflect_offset_down, height)] = reflection[:min(len(reflection)+1, height - stop_idx)]
    return np.transpose(to_ret)

In [7]:
def calculate_ieg_segm_mask(image_ts):
    reflected = reflect(image_ts)
    
    background = ndi.generic_filter(reflected, background_filter, size=50)
    skewness = skew(image_ts[image_ts>0])
    mask = image_ts > (threshold_otsu(image_ts[image_ts>0])*5/skewness + background)
    
    return threshold_otsu(image_ts[image_ts>0]), background, skewness



def calculate_ieg_segm_params(image_ts):
    
    reflected = reflect(image_ts)
    background = ndi.generic_filter(reflected, background_filter, size=PRECISION)
    skewness = skew(image_ts[image_ts>0])
    
    
    return threshold_otsu(image_ts[image_ts>0]), background, skewness

In [8]:
def ieg_segmentation(image_ts, thresh_down, thresh_up, thresh_range, mask): 

        
    elev_map = sobel(image_ts)
    distance = ndi.distance_transform_edt(mask)
    local_maxi = peak_local_max(distance, indices=False, footprint=np.ones((3,3)),
                                labels=mask)
    markers = ndi.label(local_maxi)[0]
    labels = watershed(-elev_map, markers, mask=mask)
    label_image = label(labels, connectivity = 1)
    
    
    masked = np.ma.masked_array(data = image_ts, mask = mask, fill_value = 0).filled()

    intensities_normalized = np.array([])
    dots = np.array([])
    
    for region in regionprops(label_image, intensity_image=image_ts):
        areas = np.append(areas, region.area)
        reg_app_w_bgr = pdisk(region.centroid, 7, shape = image_ts.shape)
        reg_approx = pdisk(region.centroid, 5, shape = image_ts.shape)
        reg_intensity = 2*np.sum(image_ts[reg_approx])-np.sum(image_ts[reg_app_w_bgr])
        circ = mpatches.Circle((region.centroid[1], region.centroid[0]), 5, 
                               fill=False, edgecolor=color, linewidth=2)
        new_dot = Approximation(region, copy.copy(circ), reg_intensity, len(dots))
        intensities_normalized = np.append(intensities_normalized, reg_intensity)



    return dots, intensities_normalized

In [9]:
scan_path_root = "/mnt/ula/catfish/test_scans/"

suffix = "/2.tif"
overlay_suffix = "/overlay2.tif"

In [ ]:
#MAIN loop to walk through img folder, looking for FOLDERS
results = np.array([])


for filename in os.listdir(scan_path_root):
    if os.path.isdir(scan_path_root + filename):
        print(scan_path_root + filename)
        img_path = scan_path_root + filename + suffix
        ieg_img = io.imread(img_path)
        thre, background, skewness = calculate_ieg_segm_params(ieg_img)
        res = IegImgParams(scan_path_root + filename, thre, skewness, background)
        results = np.append(results, res)

/mnt/ula/catfish/test_scans/CA1_left_glass_19_rat_16_hipp_left_skan_2019-10-11_135612
/mnt/ula/catfish/test_scans/CA1_right_glass_1_rat_3_hipp_right_skan_2019-10-08_135307
/mnt/ula/catfish/test_scans/CA1_right_glass_19_rat_14_hipp_right_skan_2019-10-11_151922
/mnt/ula/catfish/test_scans/CA1_right_glass_23_rat_20_hipp_right_skan_2019-10-14_130039
/mnt/ula/catfish/test_scans/CA1_left_glass_19_rat_13_hipp_left_skan_2019-10-11_114708
/mnt/ula/catfish/test_scans/CA1_right_glass_1_rat_2_hipp_right_skan_2019-10-08_113103
/mnt/ula/catfish/test_scans/CA1_left_glass_23_rat_17_hipp_left_skan_2019-10-14_101906
/mnt/ula/catfish/test_scans/CA1_right_glass_23_rat_19_hipp_right_skan_2019-10-14_114530
/mnt/ula/catfish/test_scans/CA1_left_glass_1_rat_3_hipp_left_skan_2019-10-08_130731
/mnt/ula/catfish/test_scans/CA1_right_glass_15_rat_11_hipp_right_skan_2019-10-11_102835
/mnt/ula/catfish/test_scans/CA1_right_glass_10_rat_6_hipp_right_skan_2019-10-09_114941
/mnt/ula/catfish/test_scans/CA1_right_glass_1_r

In [ ]:
plt.imshow(background)
plt.plot()

In [ ]:
pow(50, 2)/20

In [ ]:
np.array([[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5],[1,2,3,4,5]])[:3,:3]

In [ ]:
hist, bins = np.histogram(flattened_img, pow(PRECISION, 2)/20)